In [44]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis by computing the Gram matrix A and using A.gram_schmidt()

In [26]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[QQ(q).numerator(),QQ(q).denominator()] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R = PolynomialRing(K, 'x')
        x = R.gen()
        if n.is_integer() and (x**2-n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(x**2-n,names=gen_name)
        #NOTE: the required square roots are not all integers. some of them are algebraic numbers
        if not n.is_integer() and sqrt(n).minpoly().is_irreducible():
            gen_name = "deg" + str(n.minpoly().degree()) + "index" + str(list(set(required_square_roots)).index(n))
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [3]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [4]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [5]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
delta = lambda s: lambda t: 1 if t == s else 0 #delta function \delta_s(t)
def dft(SGA,unitary=False):
    K = containing_field(SGA)
    fourier_transform = [flatten([hat(delta(g),partition,SGA,K,unitary).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]
    if unitary:
        return matrix(K,fourier_transform).transpose()
    else:
        return matrix(fourier_transform).transpose()

In [15]:
n = 4

In [24]:
SGA = SymmetricGroupAlgebra(QQ,n); SGA

Symmetric group algebra of order 4 over Rational Field

In [25]:
K = containing_field(SGA); K

AttributeError: 'sage.rings.integer.Integer' object has no attribute 'minpoly'

In [76]:
K.base_ring()

Number Field in sqrt2 with defining polynomial x^2 - 2

In [77]:
G = SGA.group(); G

Standard permutations of 4

In [78]:
SGA_dft = SGA.dft(); SGA_dft

24 x 24 dense matrix over Rational Field (use the '.str()' method to see the entries)

In [79]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

24 x 24 dense matrix over Rational Field (use the '.str()' method to see the entries)

In [80]:
#compute the unitary DFT of the symmetric group algebra
U_dft = dft(SGA,unitary=True); U_dft

24 x 24 dense matrix over Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field (use the '.str()' method to see the entries)

In [81]:
U_dft*U_dft.conjugate().transpose()

24 x 24 dense matrix over Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field (use the '.str()' method to see the entries)

In [82]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [87]:
#QUESTION: what are the eigenvalues?
#for n=3, the minimal polynomial is degree 24 for the eigenvalues. [L:\Q] = 192, [L:K] = |Gal(L/K)| = [L:\Q]/[K:\Q] = 192/4 = 48
#there is only one subgroup of order 48 in S_6, S_2 x S_4. however, there is only one transitive permutation group, S_2 \wr S_3
#the eigenvalues are not roots of unity since min_poly(x) has rational coefficients and cyclotomic polynomials have integer coefficients

In [83]:
U_dft.charpoly()

x^24 + (1/6*sqrt2*sqrt3 - 1/6*sqrt2)*x^23 + ((-1/54*sqrt2 - 17/54)*sqrt3 + 1/12*sqrt2 - 67/162)*x^22 + ((7/324*sqrt2 - 1/108)*sqrt3 + 91/486*sqrt2 - 17/108)*x^21 + ((343/2916*sqrt2 + 923/5832)*sqrt3 - 103/972*sqrt2 - 1697/2916)*x^20 + ((-623/1944*sqrt2 - 364/2187)*sqrt3 + 1579/17496*sqrt2 + 346/729)*x^19 + ((8159/104976*sqrt2 + 40559/104976)*sqrt3 - 8027/34992*sqrt2 - 7021/26244)*x^18 + ((-5363/314928*sqrt2 + 7459/104976)*sqrt3 + 2851/34992*sqrt2 + 23315/104976)*x^17 + ((-25231/314928*sqrt2 - 15109/104976)*sqrt3 + 12361/69984*sqrt2 + 1118/6561)*x^16 + ((1063/5832*sqrt2 + 15577/69984)*sqrt3 - 89153/629856*sqrt2 - 428785/629856)*x^15 + ((-11675/52488*sqrt2 - 84617/209952)*sqrt3 + 6545/26244*sqrt2 + 45113/69984)*x^14 + ((53225/629856*sqrt2 + 41077/629856)*sqrt3 - 15437/39366*sqrt2 - 4831/209952)*x^13 + ((-53225/629856*sqrt2 - 41077/629856)*sqrt3 + 15437/39366*sqrt2 + 4831/209952)*x^11 + ((11675/52488*sqrt2 + 84617/209952)*sqrt3 - 6545/26244*sqrt2 - 45113/69984)*x^10 + ((-1063/5832*sqrt2 -

In [61]:
#ISSUE: the splitting field appears to be high degree, and the coefficients are large
if len(U_dft.charpoly().factor()) != U_dft.charpoly().degree():
    L.<a> = U_dft.charpoly().splitting_field(map=False); L
else:
    L = K

In [62]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
#the relative degree is [L:K] = [L:|Q]/[K:\Q]
if L != L.algebraic_closure():
    L_deg = L.absolute_degree(); print(L_deg)
    rel_deg = L_deg/K.absolute_degree(); print(rel_deg)

In [63]:
#getting PARI stack size error
pari.allocatemem(10^10)
pari.stacksize()

PARI stack size set to 10000000000 bytes, maximum size set to 10000007168


10000000000

In [64]:
#form relative exrtension M. attempt to compute Galois group
#for a splitting field L, we should have [L:K] = |Gal(L/K)|
if U_dft.charpoly().is_irreducible():
    M.<b> = K.extension(U_dft.charpoly()); M

In [65]:
#look at all subgroups of order [L:K] in S_d where d is the degree of the polynomial
#for n=3, [L:K] = 48 and it appears there is exactly one subgroup of order 48 up to isomorphism
subgroups_order_48_in_sym = [H for H in SymmetricGroup(6).subgroups() if H.order() == 48]
all([subgroups_order_48_in_sym[0].is_isomorphic(H) for H in subgroups_order_48_in_sym])

True

In [66]:
#one can factor the polynomial over a splitting field L/K
#but there is no way to express the roots of a quintic in terms of radicals
try:
    eigs = matrix(L,U_dft).eigenvalues(extend=False)
except TypeError:
    print("Cannot express eigenvalues in terms of radicals since polynomial is a quintic or above.")

In [84]:
eigs

[-1, 1, -0.9824456136768473? - 0.1865492325557063?*I, -0.9824456136768473? + 0.1865492325557063?*I, -0.8985365719703156? - 0.4388986543973836?*I, -0.8985365719703156? + 0.4388986543973836?*I, -0.7578648392050981? - 0.6524115920924694?*I, -0.7578648392050981? + 0.6524115920924694?*I, -0.4633311957343842? - 0.8861851968179935?*I, -0.4633311957343842? + 0.8861851968179935?*I, -0.2805792598087796? - 0.9598308595607651?*I, -0.2805792598087796? + 0.9598308595607651?*I, -0.03584488349997613? - 0.9993573656739981?*I, -0.03584488349997613? + 0.9993573656739981?*I, 0.2574814524105171? - 0.9662832409105369?*I, 0.2574814524105171? + 0.9662832409105369?*I, 0.5606995539799094? - 0.8280193295852039?*I, 0.5606995539799094? + 0.8280193295852039?*I, 0.7835078863685053? - 0.6213818407375271?*I, 0.7835078863685053? + 0.6213818407375271?*I, 0.9104280922597108? - 0.4136673649495976?*I, 0.9104280922597108? + 0.4136673649495976?*I, 0.9927583939109320? - 0.1201281454089212?*I, 0.9927583939109320? + 0.120128145

In [68]:
[arg(eig).n(20) for eig in eigs]

[3.1416,
 0.00000,
 -2.9539,
 2.9539,
 -2.6872,
 2.6872,
 -2.4308,
 2.4308,
 -2.0525,
 2.0525,
 -1.8552,
 1.8552,
 -1.6066,
 1.6066,
 -1.3104,
 1.3104,
 -0.97557,
 0.97557,
 -0.67051,
 0.67051,
 -0.42648,
 0.42648,
 -0.12042,
 0.12042]

In [86]:
eigs[4].minpoly()

x^88 + 154/81*x^86 + 4/9*x^85 + 5597/8748*x^84 + 248/2187*x^83 - 2773199/2125764*x^82 - 1897717/1417176*x^81 - 3732141845/2754990144*x^80 - 237459079/229582512*x^79 + 78024409/57395628*x^78 + 7743151099/24794911296*x^77 + 3412339717271/892616806656*x^76 + 122365220207/148769467776*x^75 + 12168324306991/8033551259904*x^74 - 6788090639645/5355700839936*x^73 - 83997881282737/42845606719488*x^72 - 1180361015890277/289207845356544*x^71 - 6330087081239947/5205741216417792*x^70 - 6218789334662/2541865828329*x^69 + 59743233908274259/20822964865671168*x^68 + 44923604592040819/31234447298506752*x^67 + 11793908356546037861/1686660154119364608*x^66 + 1889157075119241847/843330077059682304*x^65 + 540922048873901641909/121439531096594251776*x^64 - 412750056499802291/843330077059682304*x^63 - 27555164348880807001/22769912080611422208*x^62 - 891138369272084145617/182159296644891377664*x^61 - 2625232068490205412665/1092955779869348265984*x^60 - 2620420202684848659251/546477889934674132992*x^59 + 146697

In [88]:
U_dft.charpoly().discriminant()

(-1097537148561140043231077474376117315085376169988313631120122593508025406870957878250033169297046656481414070287573648234776748036784944622413484256951208851484965972491796734339520662905569127073701125173853112119356057235797/748433857324425193879278681036167099810251147438939659091104072726633308783216733399242212665264990524589475606376644299739335791035181425151853287156083409485968873884359569832575966324298088448*sqrt2 - 1118631790632044444109384410975253624900678469772373713789006044524923617061443268864331387786062161100786972673477197128733421810472945980688081630303210824374174497816968024010265405170910169005182511402750700707308497321857/1122650785986637790818918021554250649715376721158409488636656109089949963174825100098863318997897485786884213409564966449609003686552772137727779930734125114228953310826539354748863949486447132672)*sqrt3 - 18233910439386022281433760434252253484076564489205081627220214392360284723561040445075107011521463691975404653266893687613743405061714

In [ ]:
-sqrt(K(3))

In [ ]:
((-213828613/1679616*(sqrt(2)) + 85996015/69984)*(sqrt(3)) + 348251435/62208*(sqrt(2)) - 33598899709/3359232).n()

In [ ]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
SGA_dft.base_ring()

In [ ]:
SGA_eigs = SGA_dft.eigenvalues(); SGA_eigs

In [ ]:
[abs(eig) for eig in SGA_eigs]

In [ ]:
SGA_eigs[0].minpoly()

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((SGA_dft*SGA_dft.transpose()).numpy().diagonal()))